In [1]:
import numpy as np
from sklearn.metrics import roc_auc_score
def STL_AUC(model,SDG,y_true):
    Skip_Tag = []
        
    y_pred = model.predict_generator(SDG) # (#data x 50()) 
    y_pred_out = y_pred.copy()
    #print(y_pred.shape,self.y.shape,len(self.SDG.text_filenames))
    for i in range(len(y_pred)):
        for j in range(len(y_pred[i])):
            if y_pred[i][j] >= 0.5 : 
                y_pred[i][j] = 1
            else:
                y_pred[i][j] = 0
    
    #print(y_pred.shape,y_true.shape)
    
    try:
        AUC_this_epoch = roc_auc_score(y_true ,y_pred)
    except:
        AUC_this_epoch = 0
    
    
    AUC_List = []
    y_pred_T = y_pred.T
    y_true_T = y_true.T
    
    for i in range(50):
        
        try:
            AUC_List.append( roc_auc_score(y_true_T[i] ,y_pred_T[i]))
        except:
            AUC_List.append(0)
    return AUC_this_epoch,AUC_List,y_pred_out

In [2]:
##read Data Base
from pymongo import MongoClient
from bson.objectid import ObjectId
uri = "mongodb://localhost:27017/database" #mongodb://<user_name>:<user_password>@ds<xxxxxx>.mlab.com:<xxxxx>/<database_name>
conn =  MongoClient(host='localhost', port=27017, connect=False)
db = conn.PaperData
Spectrogram_Collection  =  db.get_collection('MSD_mel1366')
#LineCNN_Collection  =  db.get_collection('LineCNNMatrix_100thTag')

In [3]:
Spectrogram_Collection.count()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


58487

In [4]:
Spectrogram_Collection.create_index([("Filename","text")])

'Filename_text'

In [5]:
#Audio Para
_mel_scale = 96
_time_len = 1366
_channels = 1

In [6]:
_num_Tags = 50
_epchos = 200
_stopEpcho = 10
_batchSize = 32

In [7]:
import json 
import codecs
import numpy as np
def read_json(filename):
    with codecs.open(filename,'r',encoding = 'utf8') as infile:
        return np.array(json.load(infile))
    
x_train = read_json('New_x_train.json')
y_train = read_json('New_y_train.json')
x_test = read_json('New_x_test.json')
y_test = read_json('New_y_test.json')
x_val = read_json('New_x_val.json')
y_val = read_json('New_y_val.json')

In [8]:
len(y_test)

8486

In [9]:
from keras.layers import *
from keras.models import Model
import tensorflow as tf
from keras.models import Sequential
from keras.layers import LSTM, Dense, TimeDistributed
from keras.utils import to_categorical

Using TensorFlow backend.


In [10]:
_CL_1_kernal = 169
_CL_2_kernal = 339
_CL_3_kernal = 339
_CL_4_kernal = 339
_RNN_1_kernal = 169
_RNN_2_kernal = 169


In [11]:
Spectrogram_Input = Input(name= 'AudioInput',shape=(_mel_scale, _time_len, _channels)) #(None(1), 96, 1292, 1)
CL_1 = Conv2D(_CL_1_kernal, kernel_size=(3, 3),padding='same', name = "CL_1")(Spectrogram_Input) #(None, 96, 1292, 128) 
BN_1 = BatchNormalization()(CL_1)
AL_1 = ELU()(BN_1)
MP_1 = MaxPool2D(pool_size= (2,2), name = "MPL_1")(AL_1) #(None, 48, 323, 128)
DP_1 = Dropout(0.1)(MP_1)

CL_2 = Conv2D(_CL_2_kernal, kernel_size=(3, 3),padding='same', name = "CL_2")(DP_1) # (None, 48, 323, 384) 
BN_2 = BatchNormalization()(CL_2)
AL_2 = ELU()(BN_2)
MP_2 = MaxPool2D(pool_size= (3,3), name = "MPL_2")(AL_2) # (None, 12, 64, 384) 
DP_2 = Dropout(0.1)(MP_2)

CL_3 = Conv2D(_CL_3_kernal, kernel_size=(3, 3),padding='same', name = "CL_3")(DP_2) #(None, 12, 64, 768) 
BN_3 = BatchNormalization()(CL_3)
AL_3 = ELU()(BN_3)
MP_3 = MaxPool2D(pool_size= (4,4), name = "MPL_3")(AL_3) #(None, 4, 8, 768)
DP_3 = Dropout(0.1)(MP_3)

CL_4 = Conv2D(_CL_4_kernal, kernel_size=(3, 3),padding='same', name = "CL_4")(DP_3) #(None, 4, 8, 2048) 
BN_4 = BatchNormalization()(CL_4)
AL_4 = ELU()(BN_4)
MP_4 = MaxPool2D(pool_size= (4,4), name = "MPL_4")(AL_4)#(None, 1, 1, 2048) 
DP_4 = Dropout(0.1)(MP_4)

AR = Permute((3,2,1))(DP_4)
RS = Reshape((14, _CL_2_kernal))(DP_4)

GRU_1 = GRU(_RNN_1_kernal, return_sequences=True, name='GRU_1')(RS)
GRU_2 = GRU(_RNN_2_kernal, return_sequences=False, name='GRU_2')(GRU_1)

#pseudo-model for sheck summary
Softmax = Dense(_num_Tags, kernel_initializer='normal',activation='sigmoid')(GRU_2)
Audio_model = Model(inputs=Spectrogram_Input, outputs=Softmax)

In [12]:
Audio_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
AudioInput (InputLayer)      (None, 96, 1366, 1)       0         
_________________________________________________________________
CL_1 (Conv2D)                (None, 96, 1366, 169)     1690      
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 1366, 169)     676       
_________________________________________________________________
elu_1 (ELU)                  (None, 96, 1366, 169)     0         
_________________________________________________________________
MPL_1 (MaxPooling2D)         (None, 48, 683, 169)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 683, 169)      0         
_________________________________________________________________
CL_2 (Conv2D)                (None, 48, 683, 339)      515958    
__________

In [13]:
from keras.utils import Sequence
import json
import codecs
class AudioDataGenerator(Sequence):
    #batch size can only be 1
    def __init__(self, text_filenames, labels,batch_size):
        self.text_filenames, self.labels = text_filenames, labels
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.text_filenames)/ (self.batch_size)))

    def __getitem__(self, idx):
        #Here, you have to imprement what the data looks like in each epcho
        filename_List =  self.text_filenames[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        #Text_batch_x = [] 
        Audio_batch_x = []
        for filename in filename_List:  
            Audio_x = np.array(Spectrogram_Collection.find_one({"Filename":filename})['Spectrogram']).reshape((_mel_scale,_time_len,_channels))
            Audio_batch_x.append(Audio_x)
        y_List = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        
        return np.array(Audio_batch_x), np.array(y_List)
    def getitem(self, idx):
        #Here, you have to imprement what the data looks like in each epcho
        
        return self.__getitem__(idx)

In [14]:
ADG_train = AudioDataGenerator(x_train, y_train, _batchSize)
ADG_test = AudioDataGenerator(x_test[:8480], y_test[:8480], _batchSize) #skip 6 song while training
ADG_val = AudioDataGenerator(x_val, y_val, _batchSize)

In [15]:
## This callback is for binary softmax 

from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class AUC_callback(Callback):
    def __init__(self,y_true,SDG,batchsize):
        self.y_true = y_true
        self.SDG = SDG
        self.step = (len(y_true) // _batchSize)
        
    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        
        Skip_Tag = []
        
        y_pred = self.model.predict_generator(generator = self.SDG,
                                         steps = self.step,
                                        workers = 12,
                                        use_multiprocessing=True,) # (#data x 50()) 
        y_pred_out = y_pred.copy()
    #print(y_pred.shape,self.y.shape,len(self.SDG.text_filenames))
        for i in range(len(y_pred)):
            for j in range(len(y_pred[i])):
                if y_pred[i][j] >= 0.5 : 
                    y_pred[i][j] = 1
                else:
                    y_pred[i][j] = 0
        try:
            AUC_this_epoch = roc_auc_score(self.y_true ,y_pred)
        except Exception as e:
            AUC_this_epoch = 0
            print(e)
    
        AUC_List = []
        y_pred_T = y_pred.T
        y_true_T = self.y_true .T
    
        for i in range(50):
            try:
                AUC_List.append( roc_auc_score(y_true_T[i] ,y_pred_T[i]))
            except:
                AUC_List.append(0)

        logs['AUC_test'] = AUC_this_epoch
        
        logs['AUC_List'] = AUC_List
        
        print('AUC_test: %s '% (str(round(AUC_this_epoch,5))))
        
        
        
    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [42]:
## This callback is for binary softmax 

from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback
class AUC_callback_TF(Callback):
    def __init__(self,y_true,SDG,batchsize):
        self.y_true = y_true
        self.SDG = SDG
        self.step = (len(y_true) // _batchSize)
        
    def on_train_begin(self, logs={}):
        return

    def on_train_end(self, logs={}):
        return

    def on_epoch_begin(self, epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        
        Skip_Tag = []
        
        y_pred = self.model.predict_generator(generator = self.SDG,
                                         steps = self.step,
                                        workers = 12,
                                        use_multiprocessing=True,) # (#data x 50()) 
        y_pred_out = y_pred.copy()
    #print(y_pred.shape,self.y.shape,len(self.SDG.text_filenames))
        for i in range(len(y_pred)):
            for j in range(len(y_pred[i])):
                if y_pred[i][j] >= 0.5 : 
                    y_pred[i][j] = 1
                else:
                    y_pred[i][j] = 0
        try:
            auc, update_op = tf.metrics.auc(self.y_true ,K.round(y_pred))
            with tf.Session() as sess:
                sess.run(tf.local_variables_initializer())
                AUC_this_epoch= sess.run([auc, update_op])[0]
        except Exception as e:
            AUC_this_epoch = 0
            print(e)
    
        AUC_List = []
        y_pred_T = y_pred.T
        y_true_T = self.y_true.T
    
        for i in range(50):
            try:
                auc, update_op = tf.metrics.auc(y_true_T[i] ,K.round(y_pred_T[i]))
                with tf.Session() as sess:
                    sess.run(tf.local_variables_initializer())
                    AUC_List.append(sess.run([auc, update_op])[0])
            except Exception as e:
                AUC_List.append(0)
                print(e)
        logs['AUC_test'] = AUC_this_epoch
        
        logs['AUC_List'] = AUC_List
        
        print('AUC_test: %s '% (str(round(AUC_this_epoch,5))))
        
        
    def on_batch_begin(self, batch, logs={}):
        return

    def on_batch_end(self, batch, logs={}):
        return

In [43]:
Test_AUC = AUC_callback_TF(y_test[:8480],ADG_test,_batchSize)

In [44]:
import tensorflow as tf
auc, update_op = tf.metrics.auc(np.array([[0,0,1],[1,0,1],[0,1,0]]), np.array([[0,0,0],[0,0,0],[0,0,0]]))
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    print("tf auc: {}".format(sess.run([auc, update_op])))

tf auc: [0.5, 0.5]


In [45]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_auc',mode='max', patience=_stopEpcho, verbose=1)

In [46]:
from keras.callbacks import Callback
import time
class TimeHistory(Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.epoch_time_end = time.time()
        self.times.append(self.epoch_time_end - self.epoch_time_start)
        logs['Timer'] = self.epoch_time_end - self.epoch_time_start
time_callback = TimeHistory()


In [47]:
import keras_metrics
precision = keras_metrics.precision(label = 1)
recall = keras_metrics.recall(label = 0)

In [48]:
#https://www.quora.com/Why-does-my-convolutional-neural-network-always-produce-the-same-outputs

In [49]:
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath="TVT_Audio_CRNN_best.hdf5",monitor='val_auc',mode = 'max', verbose=1, save_best_only=True, )

In [50]:
from keras import backend as K
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true,  K.round(y_pred))[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [51]:
num_training_samples = len(x_train)
num_validation_samples = len(x_val)
num_test_samples =  len(x_test)

In [ ]:
#THis may be the final version of CRNN audio !!!
import warnings
warnings.filterwarnings('ignore')
from keras.utils import multi_gpu_model
parallel_model = multi_gpu_model(Audio_model, gpus=2)
parallel_model.compile(loss="binary_crossentropy",
                      optimizer='adam',
                      metrics=[auc])
History = parallel_model.fit_generator(
                    generator= ADG_train,
                    steps_per_epoch=(num_training_samples // _batchSize),
                    epochs= 100,
                    verbose=1,
                    validation_data= ADG_val,
                    validation_steps=(num_validation_samples // _batchSize),
                    workers=12, use_multiprocessing=True,
                    callbacks = [Test_AUC,early_stopping,time_callback, checkpointer]
                    )

Epoch 1/100


In [26]:
from keras.optimizers import Adam
opti = Adam(lr=0.002)

In [27]:
from keras.models import load_model
from keras.utils import multi_gpu_model

parallel_model = multi_gpu_model(Audio_model, gpus=2)
parallel_model.compile(loss="binary_crossentropy",
                      optimizer= opti,
                      metrics=[auc])
parallel_model.load_weights('Audio_CRNN_RE_best.hdf5')

In [33]:
#keep  training  
import warnings
warnings.filterwarnings('ignore')
History = parallel_model.fit_generator(
                    generator= ADG_train,
                    steps_per_epoch=(num_training_samples // _batchSize),
                    epochs= 100,
                    verbose=1,
                    validation_data= ADG_test,
                    validation_steps=(num_validation_samples // _batchSize),
                    workers=12, use_multiprocessing=True,
                    callbacks = [ early_stopping,time_callback, checkpointer]
                    )

Epoch 1/100
1060/1060 [==============================] - 652s 615ms/step - loss: 0.1311 - auc: 0.8346 - val_loss: 0.3873 - val_auc: 0.8045

Epoch 00001: val_auc improved from -inf to 0.80448, saving model to Audio_CRNN_RE_best.hdf5
Epoch 2/100
1060/1060 [==============================] - 655s 618ms/step - loss: 0.1370 - auc: 0.7952 - val_loss: 0.3684 - val_auc: 0.7906

Epoch 00002: val_auc did not improve from 0.80448
Epoch 3/100
1060/1060 [==============================] - 659s 621ms/step - loss: 0.1366 - auc: 0.7881 - val_loss: 0.3817 - val_auc: 0.7858

Epoch 00003: val_auc did not improve from 0.80448
Epoch 4/100
1060/1060 [==============================] - 662s 624ms/step - loss: 0.1367 - auc: 0.7845 - val_loss: 0.3637 - val_auc: 0.7833

Epoch 00004: val_auc did not improve from 0.80448
Epoch 5/100
1060/1060 [==============================] - 659s 622ms/step - loss: 0.1368 - auc: 0.7828 - val_loss: 0.3635 - val_auc: 0.7819

Epoch 00005: val_auc did not improve from 0.80448
Epoch 6/

In [37]:
from keras.models import load_model
from keras.utils import multi_gpu_model

parallel_model = multi_gpu_model(Audio_model, gpus=2)

parallel_model.load_weights('Audio_CRNN_RE_best.hdf5')

In [38]:
y_pred = parallel_model.predict_generator(ADG_test)

In [39]:
y_pred_out = y_pred.copy()
#print(y_pred.shape,self.y.shape,len(self.SDG.text_filenames))
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if y_pred[i][j] >= 0.5 : 
            y_pred[i][j] = 1
        else:
            y_pred[i][j] = 0

In [40]:
try:
    auc, update_op = tf.metrics.auc(y_test ,K.round(y_pred))
    with tf.Session() as sess:
        sess.run(tf.local_variables_initializer())
        print("tf auc: {}".format(sess.run([auc, update_op])))
except:
    AUC_this_epoch = 0

tf auc: [0.6297745, 0.6297745]


In [41]:
AUC_List = []
y_pred_T = y_pred.T
y_test_T = y_test.T
    
for i in range(50):
        
    try:
        auc, update_op = tf.metrics.auc(y_test_T[i] ,K.round(y_pred_T[i]))
        with tf.Session() as sess:
            sess.run(tf.local_variables_initializer())
            print("tf auc: {}".format(sess.run([auc, update_op])))
        
        
        AUC_List.append(auc)
        
    except:
        AUC_List.append(0)

tf auc: [0.6534231, 0.6534231]
tf auc: [0.6241743, 0.6241743]
tf auc: [0.6459072, 0.6459072]
tf auc: [0.6728258, 0.6728258]
tf auc: [0.7067504, 0.7067504]
tf auc: [0.72282284, 0.72282284]
tf auc: [0.5207818, 0.5207818]
tf auc: [0.65525174, 0.65525174]
tf auc: [0.53992045, 0.53992045]
tf auc: [0.5619868, 0.5619868]
tf auc: [0.5573579, 0.5573579]
tf auc: [0.5909789, 0.5909789]
tf auc: [0.8613825, 0.8613825]
tf auc: [0.543588, 0.543588]
tf auc: [0.5474859, 0.5474859]
tf auc: [0.54947364, 0.54947364]
tf auc: [0.6174882, 0.6174882]
tf auc: [0.6478371, 0.6478371]
tf auc: [0.54673547, 0.54673547]
tf auc: [0.64941263, 0.64941263]
tf auc: [0.6622169, 0.6622169]
tf auc: [0.52822673, 0.52822673]
tf auc: [0.5522953, 0.5522953]
tf auc: [0.5113181, 0.5113181]
tf auc: [0.5058585, 0.5058585]
tf auc: [0.74202037, 0.74202037]
tf auc: [0.58788806, 0.58788806]
tf auc: [0.5220632, 0.5220632]
tf auc: [0.6729144, 0.6729144]
tf auc: [0.6641269, 0.6641269]
tf auc: [0.5006593, 0.5006593]
tf auc: [0.5303705, 0.5

In [ ]:
#69epcho try

In [28]:
y_pred = parallel_model.predict_generator(ADG_test)

In [29]:
y_pred_out = y_pred.copy()
#print(y_pred.shape,self.y.shape,len(self.SDG.text_filenames))
for i in range(len(y_pred)):
    for j in range(len(y_pred[i])):
        if y_pred[i][j] >= 0.5 : 
            y_pred[i][j] = 1
        else:
            y_pred[i][j] = 0

In [30]:
try:
    auc, update_op = tf.metrics.auc(y_test ,K.round(y_pred))
    with tf.Session() as sess:
        sess.run(tf.local_variables_initializer())
        print("tf auc: {}".format(sess.run([auc, update_op])))
except:
    AUC_this_epoch = 0

tf auc: [0.63480747, 0.63480747]


In [31]:
AUC_List = []
y_pred_T = y_pred.T
y_test_T = y_test.T
    
for i in range(50):
        
    try:
        auc, update_op = tf.metrics.auc(y_test_T[i] ,K.round(y_pred_T[i]))
        with tf.Session() as sess:
            sess.run(tf.local_variables_initializer())
            print("tf auc: {}".format(sess.run([auc, update_op])))
        
        
        AUC_List.append(auc)
        
    except:
        AUC_List.append(0)

tf auc: [0.64767826, 0.64767826]
tf auc: [0.6292925, 0.6292925]
tf auc: [0.63888514, 0.63888514]
tf auc: [0.7164605, 0.7164605]
tf auc: [0.6813197, 0.6813197]
tf auc: [0.73677444, 0.73677444]
tf auc: [0.52957714, 0.52957714]
tf auc: [0.63481236, 0.63481236]
tf auc: [0.5514498, 0.5514498]
tf auc: [0.59014535, 0.59014535]
tf auc: [0.54988223, 0.54988223]
tf auc: [0.62775433, 0.62775433]
tf auc: [0.8274743, 0.8274743]
tf auc: [0.5794283, 0.5794283]
tf auc: [0.54832786, 0.54832786]
tf auc: [0.52247435, 0.52247435]
tf auc: [0.6200569, 0.6200569]
tf auc: [0.72143865, 0.72143865]
tf auc: [0.5867646, 0.5867646]
tf auc: [0.6463666, 0.6463666]
tf auc: [0.6057831, 0.6057831]
tf auc: [0.5394561, 0.5394561]
tf auc: [0.54549307, 0.54549307]
tf auc: [0.512792, 0.512792]
tf auc: [0.50711316, 0.50711316]
tf auc: [0.7794095, 0.7794095]
tf auc: [0.6600449, 0.6600449]
tf auc: [0.5128818, 0.5128818]
tf auc: [0.6955229, 0.6955229]
tf auc: [0.6355979, 0.6355979]
tf auc: [0.5045329, 0.5045329]
tf auc: [0.5203

In [ ]:
AUC , AUC_List , y = STL_AUC(parallel_model,ADG_test,y_test)

In [ ]:
AUC

In [33]:
AUC_List

[0.647678309820368,
 0.6292925330279041,
 0.6388851455778919,
 0.7164604686873104,
 0.6813197292833203,
 0.7367744805378355,
 0.5295770955852077,
 0.6348123855492506,
 0.5514497619492394,
 0.5901453397467135,
 0.5498822350119916,
 0.6277543583037901,
 0.8274742837131315,
 0.5794283118571738,
 0.5483279041746784,
 0.5224743072039797,
 0.6200569159083091,
 0.721438589422542,
 0.5867646209479895,
 0.6463665871141342,
 0.6057830576851759,
 0.539456079115765,
 0.545493048183569,
 0.5127919902519766,
 0.5071131454809313,
 0.7794094973789701,
 0.6600448819962347,
 0.5128818064080185,
 0.695522902135402,
 0.6355979087466176,
 0.5045329024937281,
 0.5203454708613685,
 0.628895172628173,
 0.5478566128039486,
 0.5200879850531166,
 0.7689491138887922,
 0.5867313178596478,
 0.5200768105553322,
 0.7592675866742956,
 0.5273004041213881,
 0.5347089737447586,
 0.516711456704494,
 0.5913530485782358,
 0.526366191005635,
 0.6940468623320499,
 0.5051406803146111,
 0.6863785309045494,
 0.5070539463345433,


In [ ]:
History = parallel_model.fit_generator(
                    generator= ADG_train,
                    steps_per_epoch=(num_training_samples // _batchSize),
                    epochs= 100,
                    verbose=1,
                    validation_data= ADG_test,
                    validation_steps=(num_validation_samples // _batchSize),
                    workers=12, use_multiprocessing=True,
                    callbacks = [ early_stopping,time_callback, checkpointer]
                    )

In [ ]:
from keras.models import model_from_json
json_string = parallel_model.to_json()
with codecs.open('Audio_CRNN_RE.json','w', encoding = 'utf8') as outfile:
    json.dump(json_string,outfile)
with codecs.open('Audio_CRNN_RE_History.json','w', encoding = 'utf8') as outfile:
    json.dump(History.history,outfile)

In [ ]:
parallel_model.load_weights('Audio_CRNN_RE_best.hdf5')

In [ ]:
AUC , AUC_List , y = STL_AUC(parallel_model,ADG_test,y_test)

In [ ]:
AUC

In [ ]:
AUC_List